In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn           as sb
import pandas            as pd

colours = ["windows blue", "amber", "pale red", "greyish", "faded green", "dusty purple", "orange", "turquoise", "magenta"]

sb.set()
sb.set_style("white")
sb.set_palette(sb.xkcd_palette(colours))

In [ ]:
from network_simulation import *

In [ ]:
!pip3 install --upgrade infomap

In [ ]:
import infomap

In [ ]:
# generate a network
G, hubs, attacker = generate_graph( num_nodes         =  40
                                  , num_hubs          =   3
                                  , ticks             = 250
                                  , seed              =   0
                                  , attacker_at_hub   = False
                                  , attacker_activity = 0.05
                                  , hub_fixation      = 0.25
                                  )

In [ ]:
# save the network to network.net
with open("network.net", "w") as fh:
    fh.write(serialise_graph(G, hubs, attacker))

In [ ]:
plt.matshow(networkx.adjacency_matrix(G).todense())

In [ ]:
print("hubs    ", list(hubs))
print("attacker", attacker)

In [ ]:
colours = { "regular"  : sb.color_palette()[0] # windows blue
          , "hub"      : sb.color_palette()[1] # amber
          , "attacker" : sb.color_palette()[2] # pale red
          }

pos          = { node : (G.nodes[node]["x"], G.nodes[node]["y"]) for node in G.nodes }
node_colours = []
for node in G.nodes:
    if node in hubs:
        node_colours.append(colours["hub"])
    elif node == attacker:
        node_colours.append(colours["attacker"])
    else:
        node_colours.append(colours["regular"])
edges        = G.edges
weights      = [1+log(G[u][v]["weight"]) for u,v in edges]

plt.figure(figsize=(6,6))
networkx.draw_networkx(G, pos = pos, node_color = node_colours, width = weights)

In [ ]:
# plot the degrees per node
xys = dict(G.degree)
plt.scatter(xys.keys(), xys.values(), marker = "x")
plt.hlines(average(list(xys.values())), xmin=0, xmax=len(xys))
plt.xlim(0, len(xys))

In [ ]:
# run infomap
im = infomap.Infomap()
for (u,v) in G.edges:
    im.add_link(u, v, G.edges[(u,v)]["weight"])
im.run()

In [ ]:
# print the cluster structure and mark the attacker
for node in im.tree:
    if node.is_leaf:
        if node.node_id == attacker:
            print(">> ", end="")
        print(f"{node.node_id} {node.module_id} {node.flow:0.2f}")